In [48]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [62]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [63]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
#tx_clean = remove_undefined_values (tX)
ind_remov

0.152456
0.0
0.0
0.0
0.709828
0.709828
0.709828
0.0
0.0
0.0
0.0
0.0
0.709828
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.399652
0.399652
0.399652
0.709828
0.709828
0.709828
0.0


[4, 5, 6, 12, 26, 27, 28]

In [106]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
y, tx = build_model_data(tx,y)

y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [107]:
num_samples, num_features
tx.shape, tx

((250000, 24), array([[ 1.        ,  0.52692419,  0.06833197, ...,  0.82689468,
          0.81821738,  0.4125105 ],
        [ 1.        ,  0.80440808,  0.55250482, ...,  0.81960824,
          0.8218683 , -0.27381996],
        [ 1.        , -1.78875255,  3.19515553, ...,  0.82639869,
          0.80557722, -0.29396985],
        ...,
        [ 1.        ,  0.11918948,  0.31931645, ...,  0.82510503,
          0.81509825, -0.31701723],
        [ 1.        , -0.01056731, -0.84532397, ..., -1.22560263,
         -1.22560163, -0.74543941],
        [ 1.        , -1.78875255,  0.66533608, ..., -1.22560263,
         -1.22560163, -0.74543941]]))

In [108]:
from compute_gradient import *
from cost import *
from implementations import *

In [109]:
#Initial weight


#Maximum iterations through the whole set
max_iter = 1000

#0.001 nan
#0.0005 nan

#Step-size
gamma = 0.001
gammas = np.logspace(-6,-3, 7)
lambdas = np.logspace (-5,0,7)
degrees = 7

#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
lambda_ = 2e-2
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 24), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [ ]:
loss_val_tot = []
loss_train_tot = []
k_fold = 3
loss_min = np.inf

for ind, degree in enumerate(range(1,degrees+1)):
    tx = build_poly_all_features(tx,degree)
    y,tx = build_model_data(tx,y)
    loss_training = 0
    loss_validation = 0
    for i,k in enumerate(range(k_fold)):
        initial_w = np.zeros((tx.shape[1],1))
        tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
        w, loss_train_k = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iter, gamma)
        loss_val_k = reg_logistic_loss (y_val,tx_val,w,lambda_)/len(y_val)
        loss_train_k /= len(y_train)
        loss_training += loss_train_k
        loss_validation += loss_val_k
        
        
    loss_training /= k_fold
    loss_validation /= k_fold
    if ((loss_training+loss_validation)/2.0) < loss_min:
        weights_star = w
        degree_star = degree
        loss_min = (loss_training+loss_val)/2.0
        print(loss_min)
    loss_val_tot.append(loss_validation)
    loss_train_tot.append(loss_training)

plot_cross_validation(degrees, "Lambda", degree_star, loss_train_tot, loss_val_tot, loss)
print(" Best value of degree = {g} \n Loss = {l} \n Weights = {we}".format(
    g=degree_star, l=loss, we = weights_star))

In [ ]:
#PREDICT TRAINING ACCURACY
y_pred_training = predict_labels(weights_star, tx,'logistic')
count = 0
for i in range(num_samples):
    if (y_pred_training[i] == -1 and y[i] == 0):
        count += 1
    elif (y_pred_training[i] == y[i]):
        count += 1
accuracy = (count *100) / num_samples

accuracy

In [10]:
losses_tr = np.zeros((len(gammas),len(lambdas)))
losses_val = np.zeros((len(gammas),len(lambdas)))

k_fold = 2
seed = 1

for ind_lambda, lambda_ in enumerate(lambdas):
    for ind_gamma ,gamma_ in enumerate (gammas):
        for ind_degree, degree in enumerate (degrees):
            tx = build_poly_all_features(tx,degree)
            for i,k in enumerate(range(k_fold)):
                
                tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
                w, loss_training = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iter, gamma)
                loss_val = reg_logistic_loss (y_val,tx_val,w,lambda_)/len(y_val)
                loss_training /= len(y_train)
                losses_tr[ind_lambda][ind_gamma]= loss_training
                losses_val [ind_lambda][ind_gamma] = loss_val
            
            
                if loss_training < loss_min:
                    weights_star = w
                    gamma_star = gamma
                    lambda_star = lambda_
                    loss_min = loss_training

109756.20152532344
106358.24427164088
103998.8971821258
101962.78875222043
100254.79069723404
99122.38969705586
98024.67377478634
96873.45443026749
96200.26310447392
95512.88918771365
94633.74422621673
94174.9225446923
93700.44105454849
92984.60449754844
92635.08271871509
92283.37906065678
91681.76554292584
91390.07583801802
91114.97847155234
90602.76483372312
90341.48166176202
90117.24236954766
89680.53116548403
89434.13306026228
89245.18785906598
88875.1919369561
88634.3119818995
88470.7695883172
88161.42409864564
87920.26124928558
87775.4829514335
87522.16153524394
87285.51012460973
87146.38468264748
86945.38989317586


/Users/Sofia/Desktop/ML_Project1/cost.py:20: RuntimeWarning: divide by zero encountered in log
  loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))


inf
86587.41363136146
86422.20268752072
nan
nan
85945.01798911016
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
inf
nan
nan
84740.28664937796
nan
nan
87838.05540500928
nan
nan
90451.7818387172
nan
91773.01016959178
92466.24412926171
93031.61730846028
93469.53342641865
93943.10752045868
94390.89359130901
94689.38939216967
94996.1247354117
95353.57864514536
95550.10579544837
95735.97691935896
96027.5590714678


In [11]:
losses_tr

array([[0.62399588, 0.59473672, 0.57720389, 0.5650518 , 0.55581272,
        0.54834265, 0.54205106],
       [0.53660694, 0.531808  , 0.52752368, 0.52371516,        inf,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan, 0.55819194, 0.56634763,
        0.57212376, 0.57616766]])

In [12]:
losses_val

array([[0.62358287, 0.59419693, 0.57664055, 0.56451943, 0.55534669,
        0.54796646, 0.54177972],
       [0.53644946, 0.53176947, 0.52760732, 0.52387337, 0.52050458,
        0.5174515 , 0.51467054],
       [0.51213306, 0.50980579, 0.50766861, 0.50570216, 0.50388547,
        0.50219951, 0.50062936],
       [0.49920404, 0.49786857, 0.49661534, 0.4954376 , 0.49432922,
        0.49328466, 0.49229889],
       [0.49162306, 0.49097199, 0.49034576, 0.48974414, 0.48916671,
        0.48861291, 0.48808202],
       [0.48935751, 0.49060756, 0.49181166, 0.49296242, 0.49405869,
        0.49510162, 0.49609291],
       [0.5151914 , 0.53307717, 0.54757207, 0.55853499, 0.56649348,
        0.57213713, 0.57609077]])

In [37]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [38]:
#CLEAN AND STANDARDIZE TEST SET
tx_test_clean = remove_undefined_values (tX_test)
tx_test, _, _ = standardize(tx_test_clean,0)
tx_test = build_poly_all_features(tx_test,2)

In [39]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test,ytest)

In [41]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-IMPORTANT.csv'
y_pred = predict_labels(w, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [31]:
lambda_star, gamma_star,loss_min

(1.0, 5e-05, inf)

In [23]:
gammas

array([1.00000000e-06, 3.16227766e-06, 1.00000000e-05, 3.16227766e-05,
       1.00000000e-04, 3.16227766e-04, 1.00000000e-03])

In [24]:
lambdas

array([1.00000000e-05, 6.81292069e-05, 4.64158883e-04, 3.16227766e-03,
       2.15443469e-02, 1.46779927e-01, 1.00000000e+00])

In [25]:
(losses_tr+losses_val)/2

array([[0.62378938, 0.59446682, 0.57692222, 0.56478561, 0.5555797 ,
        0.54815455, 0.54191539],
       [0.5365282 , 0.53178873, 0.5275655 , 0.52379426,        inf,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan],
       [       nan,        nan,        nan, 0.55836346, 0.56642055,
        0.57213044, 0.57612922]])